### Chapter 15: Interfaces and Modports

#### Introduction

Interfaces are one of SystemVerilog's most powerful features for creating reusable, hierarchical designs. They encapsulate communication protocols between modules, making designs more modular, readable, and maintainable. Modports define directional views of interfaces, ensuring proper connectivity and access control.

#### Interface Declarations

An interface is a named bundle of signals that can be shared between modules. It acts as a communication channel that groups related signals together.

##### Basic Interface Syntax

```systemverilog
interface interface_name [parameter_list] [port_list];
    // Signal declarations
    // Always blocks
    // Tasks and functions
    // Modport declarations
endinterface
```

##### Simple Interface Example

```systemverilog
// Basic memory interface
interface memory_if;
    logic [31:0] addr;
    logic [31:0] data;
    logic        we;     // write enable
    logic        re;     // read enable
    logic        ready;
    logic        valid;
endinterface
```

##### Interface with Clock and Reset

```systemverilog
interface cpu_bus_if (input logic clk, input logic rst_n);
    logic [31:0] addr;
    logic [31:0] wdata;
    logic [31:0] rdata;
    logic [3:0]  be;      // byte enable
    logic        req;
    logic        ack;
    logic        we;
    
    // Clocking block for synchronous operations
    clocking cb @(posedge clk);
        default input #1step output #0;
        output addr, wdata, be, req, we;
        input  rdata, ack;
    endclocking
    
    // Reset logic
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            req <= 1'b0;
        end
    end
endinterface
```

#### Modport Definitions

Modports define different views of an interface, specifying signal directions from the perspective of different modules. They provide access control and improve code readability.

##### Basic Modport Syntax

```systemverilog
modport modport_name (
    input  signal_list,
    output signal_list,
    inout  signal_list,
    ref    signal_list,
    import task_function_list
);
```

##### Memory Interface with Modports

```systemverilog
interface memory_if;
    logic [31:0] addr;
    logic [31:0] wdata;
    logic [31:0] rdata;
    logic        we;
    logic        re;
    logic        ready;
    logic        valid;
    
    // Master modport (CPU side)
    modport master (
        output addr, wdata, we, re,
        input  rdata, ready
    );
    
    // Slave modport (Memory side)
    modport slave (
        input  addr, wdata, we, re,
        output rdata, ready
    );
    
    // Monitor modport (for verification)
    modport monitor (
        input addr, wdata, rdata, we, re, ready
    );
    
    // Tasks accessible through modports
    task write_data(input [31:0] address, input [31:0] data);
        @(posedge clk);
        addr  = address;
        wdata = data;
        we    = 1'b1;
        re    = 1'b0;
        wait(ready);
        @(posedge clk);
        we = 1'b0;
    endtask
    
    modport master_with_tasks (
        output addr, wdata, we, re,
        input  rdata, ready,
        import write_data
    );
endinterface
```

#### Interface Instantiation

Interfaces are instantiated like modules and can be connected to multiple modules simultaneously.

##### Module Using Interface

```systemverilog
// CPU module using memory interface
module cpu (
    input logic clk,
    input logic rst_n,
    memory_if.master mem_bus  // Using master modport
);
    
    typedef enum logic [2:0] {
        IDLE, FETCH, DECODE, EXECUTE, WRITEBACK
    } cpu_state_t;
    
    cpu_state_t state, next_state;
    logic [31:0] pc;
    logic [31:0] instruction;
    
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            state <= IDLE;
            pc <= 32'h0;
        end else begin
            state <= next_state;
        end
    end
    
    always_comb begin
        next_state = state;
        mem_bus.addr = 32'h0;
        mem_bus.wdata = 32'h0;
        mem_bus.we = 1'b0;
        mem_bus.re = 1'b0;
        
        case (state)
            IDLE: begin
                next_state = FETCH;
            end
            
            FETCH: begin
                mem_bus.addr = pc;
                mem_bus.re = 1'b1;
                if (mem_bus.ready) begin
                    instruction = mem_bus.rdata;
                    pc = pc + 4;
                    next_state = DECODE;
                end
            end
            
            DECODE: begin
                // Decode logic here
                next_state = EXECUTE;
            end
            
            EXECUTE: begin
                // Execute logic here
                next_state = WRITEBACK;
            end
            
            WRITEBACK: begin
                // Writeback logic here
                next_state = FETCH;
            end
        endcase
    end
endmodule

// Memory module using interface
module memory (
    input logic clk,
    input logic rst_n,
    memory_if.slave mem_bus  // Using slave modport
);
    
    logic [31:0] mem_array [0:1023];
    logic [9:0]  addr_index;
    
    assign addr_index = mem_bus.addr[11:2];  // Word addressing
    
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            mem_bus.ready <= 1'b0;
            mem_bus.rdata <= 32'h0;
        end else begin
            mem_bus.ready <= 1'b0;
            
            if (mem_bus.we) begin
                mem_array[addr_index] <= mem_bus.wdata;
                mem_bus.ready <= 1'b1;
            end else if (mem_bus.re) begin
                mem_bus.rdata <= mem_array[addr_index];
                mem_bus.ready <= 1'b1;
            end
        end
    end
endmodule
```

##### Top-level Module with Interface

```systemverilog
module top;
    logic clk = 0;
    logic rst_n;
    
    // Clock generation
    always #5 clk = ~clk;
    
    // Reset generation
    initial begin
        rst_n = 0;
        #20 rst_n = 1;
    end
    
    // Interface instantiation
    memory_if mem_bus();
    
    // Module instantiations
    cpu cpu_inst (
        .clk(clk),
        .rst_n(rst_n),
        .mem_bus(mem_bus.master)
    );
    
    memory mem_inst (
        .clk(clk),
        .rst_n(rst_n),
        .mem_bus(mem_bus.slave)
    );
    
    // Monitor for verification
    initial begin
        $monitor("Time:%0t, Addr:%h, WData:%h, RData:%h, WE:%b, RE:%b, Ready:%b",
                 $time, mem_bus.addr, mem_bus.wdata, mem_bus.rdata,
                 mem_bus.we, mem_bus.re, mem_bus.ready);
    end
endmodule
```

#### Parameterized Interfaces

Interfaces can be parameterized to create flexible, reusable communication protocols.

##### Parameterized Bus Interface

```systemverilog
interface axi_if #(
    parameter int ADDR_WIDTH = 32,
    parameter int DATA_WIDTH = 32,
    parameter int ID_WIDTH   = 4
) (
    input logic aclk,
    input logic aresetn
);
    
    // Write Address Channel
    logic [ID_WIDTH-1:0]     awid;
    logic [ADDR_WIDTH-1:0]   awaddr;
    logic [7:0]              awlen;
    logic [2:0]              awsize;
    logic [1:0]              awburst;
    logic                    awvalid;
    logic                    awready;
    
    // Write Data Channel
    logic [DATA_WIDTH-1:0]   wdata;
    logic [DATA_WIDTH/8-1:0] wstrb;
    logic                    wlast;
    logic                    wvalid;
    logic                    wready;
    
    // Write Response Channel
    logic [ID_WIDTH-1:0]     bid;
    logic [1:0]              bresp;
    logic                    bvalid;
    logic                    bready;
    
    // Read Address Channel
    logic [ID_WIDTH-1:0]     arid;
    logic [ADDR_WIDTH-1:0]   araddr;
    logic [7:0]              arlen;
    logic [2:0]              arsize;
    logic [1:0]              arburst;
    logic                    arvalid;
    logic                    arready;
    
    // Read Data Channel
    logic [ID_WIDTH-1:0]     rid;
    logic [DATA_WIDTH-1:0]   rdata;
    logic [1:0]              rresp;
    logic                    rlast;
    logic                    rvalid;
    logic                    rready;
    
    // Master modport
    modport master (
        output awid, awaddr, awlen, awsize, awburst, awvalid,
        input  awready,
        output wdata, wstrb, wlast, wvalid,
        input  wready,
        input  bid, bresp, bvalid,
        output bready,
        output arid, araddr, arlen, arsize, arburst, arvalid,
        input  arready,
        input  rid, rdata, rresp, rlast, rvalid,
        output rready
    );
    
    // Slave modport
    modport slave (
        input  awid, awaddr, awlen, awsize, awburst, awvalid,
        output awready,
        input  wdata, wstrb, wlast, wvalid,
        output wready,
        output bid, bresp, bvalid,
        input  bready,
        input  arid, araddr, arlen, arsize, arburst, arvalid,
        output arready,
        output rid, rdata, rresp, rlast, rvalid,
        input  rready
    );
    
    // Clocking blocks for verification
    clocking master_cb @(posedge aclk);
        default input #1step output #0;
        output awid, awaddr, awlen, awsize, awburst, awvalid;
        input  awready;
        output wdata, wstrb, wlast, wvalid;
        input  wready;
        input  bid, bresp, bvalid;
        output bready;
        output arid, araddr, arlen, arsize, arburst, arvalid;
        input  arready;
        input  rid, rdata, rresp, rlast, rvalid;
        output rready;
    endclocking
    
    modport master_tb (clocking master_cb);
endinterface
```

##### Using Parameterized Interface

```systemverilog
module axi_master #(
    parameter int ADDR_WIDTH = 32,
    parameter int DATA_WIDTH = 64
) (
    axi_if.master axi_bus
);
    // Implementation using the parameterized interface
    // The interface parameters are automatically matched
endmodule

module system_top;
    logic clk = 0;
    logic rst_n;
    
    always #5 clk = ~clk;
    
    // Instantiate parameterized interface
    axi_if #(
        .ADDR_WIDTH(32),
        .DATA_WIDTH(64),
        .ID_WIDTH(8)
    ) axi_bus (
        .aclk(clk),
        .aresetn(rst_n)
    );
    
    // Connect master with matching parameters
    axi_master #(
        .ADDR_WIDTH(32),
        .DATA_WIDTH(64)
    ) master_inst (
        .axi_bus(axi_bus.master)
    );
endmodule
```

#### Interface Arrays

SystemVerilog supports arrays of interfaces, useful for multi-port designs or interconnect fabrics.

##### Interface Array Declaration

```systemverilog
interface simple_bus_if;
    logic [31:0] addr;
    logic [31:0] data;
    logic        valid;
    logic        ready;
    
    modport master (output addr, data, valid, input ready);
    modport slave  (input addr, data, valid, output ready);
endinterface

module multi_port_memory (
    input logic clk,
    input logic rst_n,
    simple_bus_if.slave ports [4]  // Array of 4 interface instances
);
    
    logic [31:0] memory [0:1023];
    
    // Handle each port independently
    genvar i;
    generate
        for (i = 0; i < 4; i++) begin : port_handler
            always_ff @(posedge clk) begin
                if (ports[i].valid) begin
                    if (ports[i].addr[31]) begin
                        // Write operation
                        memory[ports[i].addr[11:2]] <= ports[i].data;
                    end else begin
                        // Read operation
                        ports[i].data <= memory[ports[i].addr[11:2]];
                    end
                    ports[i].ready <= 1'b1;
                end else begin
                    ports[i].ready <= 1'b0;
                end
            end
        end
    endgenerate
endmodule
```

##### Multi-Master System

```systemverilog
module multi_master_system;
    logic clk = 0;
    logic rst_n;
    
    always #5 clk = ~clk;
    
    // Array of interfaces
    simple_bus_if bus_array [4]();
    
    // Multiple masters
    genvar i;
    generate
        for (i = 0; i < 4; i++) begin : masters
            cpu_core #(.CORE_ID(i)) cpu_inst (
                .clk(clk),
                .rst_n(rst_n),
                .bus(bus_array[i].master)
            );
        end
    endgenerate
    
    // Shared memory with multiple ports
    multi_port_memory memory_inst (
        .clk(clk),
        .rst_n(rst_n),
        .ports(bus_array)  // Pass entire array
    );
endmodule
```

#### Virtual Interfaces

Virtual interfaces provide a mechanism to access interface handles in classes, enabling dynamic interface access in object-oriented testbenches.

##### Virtual Interface in Classes

```systemverilog
interface spi_if (input logic clk);
    logic       cs_n;
    logic       sclk;
    logic       mosi;
    logic       miso;
    
    modport master (output cs_n, sclk, mosi, input miso);
    modport slave  (input cs_n, sclk, mosi, output miso);
    
    task send_byte(input [7:0] data);
        cs_n = 1'b0;
        for (int i = 7; i >= 0; i--) begin
            @(posedge clk);
            sclk = 1'b0;
            mosi = data[i];
            @(posedge clk);
            sclk = 1'b1;
        end
        @(posedge clk);
        cs_n = 1'b1;
    endtask
    
    modport master_with_tasks (
        output cs_n, sclk, mosi,
        input miso,
        import send_byte
    );
endinterface

// Driver class using virtual interface
class spi_driver;
    virtual spi_if.master_with_tasks vif;
    
    function new(virtual spi_if.master_with_tasks vif);
        this.vif = vif;
    endfunction
    
    task drive_transaction(input [7:0] data);
        vif.send_byte(data);
    endtask
    
    task reset_interface();
        vif.cs_n = 1'b1;
        vif.sclk = 1'b0;
        vif.mosi = 1'b0;
    endtask
endclass

// Monitor class
class spi_monitor;
    virtual spi_if.slave vif;
    
    function new(virtual spi_if.slave vif);
        this.vif = vif;
    endfunction
    
    task monitor_transactions();
        logic [7:0] received_data;
        
        forever begin
            @(negedge vif.cs_n);  // Wait for transaction start
            received_data = 8'h0;
            
            for (int i = 7; i >= 0; i--) begin
                @(posedge vif.sclk);
                received_data[i] = vif.mosi;
            end
            
            $display("Monitor: Received data = 0x%02h at time %0t", 
                     received_data, $time);
        end
    endtask
endclass
```

##### Testbench Using Virtual Interfaces

```systemverilog
module spi_testbench;
    logic clk = 0;
    logic rst_n;
    
    always #5 clk = ~clk;
    
    // Interface instantiation
    spi_if spi_bus(clk);
    
    // DUT instantiation
    spi_slave dut (
        .clk(clk),
        .rst_n(rst_n),
        .spi(spi_bus.slave)
    );
    
    // Testbench components
    spi_driver driver;
    spi_monitor monitor;
    
    initial begin
        // Create driver and monitor with virtual interfaces
        driver = new(spi_bus.master_with_tasks);
        monitor = new(spi_bus.slave);
        
        // Reset sequence
        rst_n = 0;
        driver.reset_interface();
        #20 rst_n = 1;
        
        // Start monitor
        fork
            monitor.monitor_transactions();
        join_none
        
        // Drive test transactions
        #100;
        driver.drive_transaction(8'hA5);
        #50;
        driver.drive_transaction(8'h3C);
        #50;
        driver.drive_transaction(8'hFF);
        
        #200 $finish;
    end
endmodule
```

#### Advanced Interface Concepts

##### Interface with Assertions

```systemverilog
interface protocol_if (input logic clk, input logic rst_n);
    logic       req;
    logic       ack;
    logic [7:0] data;
    logic       valid;
    
    // Protocol assertions
    property req_ack_protocol;
        @(posedge clk) disable iff (!rst_n)
        req |-> ##[1:5] ack;
    endproperty
    
    property valid_data;
        @(posedge clk) disable iff (!rst_n)
        valid |-> (data !== 8'hxx);
    endproperty
    
    assert_req_ack: assert property (req_ack_protocol)
        else $error("Request not acknowledged within 5 cycles");
    
    assert_valid_data: assert property (valid_data)
        else $error("Invalid data when valid is asserted");
    
    // Coverage
    covergroup protocol_cg @(posedge clk);
        req_cp: coverpoint req;
        ack_cp: coverpoint ack;
        data_cp: coverpoint data {
            bins low = {[0:63]};
            bins high = {[64:127]};
            bins max = {[128:255]};
        }
        
        req_ack_cross: cross req_cp, ack_cp;
    endgroup
    
    protocol_cg cg_inst = new();
endinterface
```

#### Best Practices for Interfaces

1. **Use Modports Consistently**: Always define appropriate modports to clarify signal directions and access rights.
2. **Parameterize for Reusability**: Make interfaces parameterizable for width and other configurable aspects.
3. **Include Protocol Tasks**: Embed common protocol operations as tasks within interfaces.
4. **Add Assertions**: Include protocol checking and coverage within interfaces.
5. **Hierarchical Interface Design**: Use interfaces at appropriate abstraction levels.

```systemverilog
// Good practice: Hierarchical interface design
interface chip_level_if;
    cpu_bus_if cpu_bus();
    memory_if  mem_bus();
    peripheral_if periph_bus[8]();
    
    // Interconnect logic
    always_comb begin
        // Address decoding and routing
        case (cpu_bus.addr[31:28])
            4'h0: begin
                // Route to memory
                mem_bus.addr = cpu_bus.addr;
                mem_bus.wdata = cpu_bus.wdata;
                // ... more connections
            end
            4'h1: begin
                // Route to peripherals
                // ... peripheral routing logic
            end
        endcase
    end
endinterface
```

#### Summary

Interfaces and modports are fundamental to modern SystemVerilog design methodology. They provide:

- **Modularity**: Clean separation of communication protocols from module implementation
- **Reusability**: Parameterized interfaces can be reused across different designs
- **Maintainability**: Changes to protocols are localized to interface definitions
- **Verification**: Virtual interfaces enable sophisticated testbench architectures
- **Protocol Checking**: Built-in assertions and coverage for protocol validation

Understanding and effectively using interfaces is crucial for creating scalable, maintainable SystemVerilog designs and verification environments.